In [8]:
import os
import pandas as pd
from PIL import Image
import numpy as np


run_ichallenge_amd  = True
run_ichallenge_non_amd  = True
run_ichallenge_glaucoma = True
run_ichallenge_unlabelled = True



In [9]:
def generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=None):
    
    # have to call for each dataset that contains images + masks
    # finding image (.image_postfix) and mask (.mask_postfix) pairs for same base filename
    # this works if your_img_dir/A0001.jpg and your_mask_dir/A0001.bmp
    # optic nerve head
    # converts all the masks - 0 background, 255 foreground if new_mask_path is not None
    
    mode = []
    
    # need one file for each concept 
    image_path = list(set(os.listdir(images_path)) - {'desktop.ini'})
    image_id = [os.path.basename(x).replace(image_postfix, "") for x in image_path]
    image_path = [os.path.join(images_path, x) for x in image_path]
    a = {"image_path" : image_path, "id" : image_id}
    a = pd.DataFrame(data={"img_path" : image_path}, index=image_id)
    
    has_roi = []
    
    if masks_path is not None:
        mask_path  = list(set(os.listdir(masks_path)) - {'desktop.ini'})
        mask_id = [os.path.basename(x).replace(mask_postfix, "") for x in mask_path]
        mask_path = [os.path.join(masks_path, x) for x in mask_path]
        
        new_mask_list = []
        if new_mask_path is not None:
            
            for mp in mask_path:

                # all classes = 255 (will become 1), background = 0
                mask = Image.open(mp).convert('L')
                mask = mask.point( lambda p: 0 if p == 255 else 255 )
                mask = mask.convert('1')
                
                if np.asarray(mask, dtype="int32").sum() == 0:
                    has_roi.append(False)
                else:
                    has_roi.append(True)
                
                c = os.path.join(new_mask_path, os.path.basename(mp))
                new_mask_list.append(c)
                mask.save(c)
                #mask.show()
                #break
                #print("unique mask", np.unique(mask))
                #print(self.csv_data.iloc[index]['msk_path'])
                #print("unique mask", np.unique(np.asarray(mask)))
                
                
            mask_path = new_mask_list
                
        # b = {"mask_path" : mask_path, "id" : mask_id, "has_roi" : has_roi}
        b = pd.DataFrame(data={"msk_path" : mask_path, "has_roi" : has_roi}, index=mask_id)
        csv_data = pd.concat([a, b], axis=1, join="inner")
        
        csv_data["has_mask"] = [True] * len(a.index)
        
    else:
        csv_data = a
        csv_data["mask_path"] = [None] * len(a.index)
        csv_data["has_mask"] = [False] * len(a.index)
        
    
    csv_data = csv_data.sample(frac=1, random_state=19)

    percent_train = int(len(csv_data)/100*split_percentage)
    percent_val = len(csv_data)-percent_train
    
    mode.extend(["train" for i in range(percent_train)])
    mode.extend(["val" for i in range(percent_val)])
    
    
    csv_data["mode"] = mode
    csv_data["weight"] = [weight]*len(mode)

    print(csv_data)
    
    csv_data.to_csv(csv_path, sep=';')
            

In [10]:
if run_ichallenge_amd:
    images_path =    r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\images_AMD"
    masks_path  =    r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\Disc_Masks"
    new_masks_path = r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\Disc_Masks_bin"
    if not os.path.exists(new_masks_path):
        os.makedirs(new_masks_path)
    csv_path    = r"mt_data_ichallenge_amd.csv"
    weight = 1
    generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=new_masks_path)

                                                img_path  \
A0034  C:\Users\Prinzessin\projects\image_data\iChall...   
A0039  C:\Users\Prinzessin\projects\image_data\iChall...   
A0047  C:\Users\Prinzessin\projects\image_data\iChall...   
A0069  C:\Users\Prinzessin\projects\image_data\iChall...   
A0031  C:\Users\Prinzessin\projects\image_data\iChall...   
...                                                  ...   
A0022  C:\Users\Prinzessin\projects\image_data\iChall...   
A0016  C:\Users\Prinzessin\projects\image_data\iChall...   
A0082  C:\Users\Prinzessin\projects\image_data\iChall...   
A0013  C:\Users\Prinzessin\projects\image_data\iChall...   
A0008  C:\Users\Prinzessin\projects\image_data\iChall...   

                                                msk_path  has_roi  has_mask  \
A0034  C:\Users\Prinzessin\projects\image_data\iChall...     True      True   
A0039  C:\Users\Prinzessin\projects\image_data\iChall...     True      True   
A0047  C:\Users\Prinzessin\projects\image_

In [11]:

if run_ichallenge_glaucoma:
    images_path = r"C:\Users\Prinzessin\projects\image_data\iChallenge_Glaucoma_OD_Fovea\images_Glaucoma"
    masks_path  = r"C:\Users\Prinzessin\projects\image_data\iChallenge_Glaucoma_OD_Fovea\Disc_Cup_Masks\Glaucoma"
    new_masks_path  = r"C:\Users\Prinzessin\projects\image_data\iChallenge_Glaucoma_OD_Fovea\Disc_Masks_bin"
    if not os.path.exists(new_masks_path):
        os.makedirs(new_masks_path)
    csv_path    = r"mt_data_ichallenge_glaucoma.csv"
    weight = 0.1
    generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=new_masks_path)

                                                img_path  \
g0025  C:\Users\Prinzessin\projects\image_data\iChall...   
g0017  C:\Users\Prinzessin\projects\image_data\iChall...   
g0013  C:\Users\Prinzessin\projects\image_data\iChall...   
g0007  C:\Users\Prinzessin\projects\image_data\iChall...   
g0019  C:\Users\Prinzessin\projects\image_data\iChall...   
g0029  C:\Users\Prinzessin\projects\image_data\iChall...   
g0016  C:\Users\Prinzessin\projects\image_data\iChall...   
g0020  C:\Users\Prinzessin\projects\image_data\iChall...   
g0035  C:\Users\Prinzessin\projects\image_data\iChall...   
g0005  C:\Users\Prinzessin\projects\image_data\iChall...   
g0027  C:\Users\Prinzessin\projects\image_data\iChall...   
g0024  C:\Users\Prinzessin\projects\image_data\iChall...   
g0021  C:\Users\Prinzessin\projects\image_data\iChall...   
g0022  C:\Users\Prinzessin\projects\image_data\iChall...   
g0023  C:\Users\Prinzessin\projects\image_data\iChall...   
g0033  C:\Users\Prinzessin\projects\imag

In [11]:
if run_ichallenge_non_amd:
    images_path = r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\images_AMD"
    masks_path  = r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\Disc_Masks"
    new_masks_path = r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\Disc_Masks_bin"
    if not os.path.exists(new_masks_path):
        os.makedirs(new_masks_path)
    csv_path    = r"mt_data_ichallenge_non_amd.csv"
    weight = 0.1
    generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=90, new_mask_path=new_masks_path)

                                                img_path  \
A0029  C:\Users\Prinzessin\projects\image_data\iChall...   
A0033  C:\Users\Prinzessin\projects\image_data\iChall...   
A0007  C:\Users\Prinzessin\projects\image_data\iChall...   
A0030  C:\Users\Prinzessin\projects\image_data\iChall...   
A0043  C:\Users\Prinzessin\projects\image_data\iChall...   
...                                                  ...   
A0022  C:\Users\Prinzessin\projects\image_data\iChall...   
A0062  C:\Users\Prinzessin\projects\image_data\iChall...   
A0055  C:\Users\Prinzessin\projects\image_data\iChall...   
A0041  C:\Users\Prinzessin\projects\image_data\iChall...   
A0042  C:\Users\Prinzessin\projects\image_data\iChall...   

                                                msk_path  has_mask   mode  \
A0029  C:\Users\Prinzessin\projects\image_data\iChall...      True  train   
A0033  C:\Users\Prinzessin\projects\image_data\iChall...      True  train   
A0007  C:\Users\Prinzessin\projects\image_data\i

In [12]:
if run_ichallenge_unlabelled:
    images_path = r"C:\Users\Prinzessin\projects\image_data\iChallenge_AMD_OD_Fovea_lesions\ADAM_400_testset"
    masks_path  = None
    csv_path    = "mt_data_ichallenge_unlabelled.csv"
    weight = 0
    generate_semisupseg_csv_file(images_path, masks_path, csv_path, weight, image_postfix=".jpg", mask_postfix=".bmp", split_percentage=100)

                                                img_path mask_path  has_mask  \
V0277  C:\Users\Prinzessin\projects\image_data\iChall...      None     False   
V0191  C:\Users\Prinzessin\projects\image_data\iChall...      None     False   
V0028  C:\Users\Prinzessin\projects\image_data\iChall...      None     False   
V0061  C:\Users\Prinzessin\projects\image_data\iChall...      None     False   
V0398  C:\Users\Prinzessin\projects\image_data\iChall...      None     False   
...                                                  ...       ...       ...   
V0310  C:\Users\Prinzessin\projects\image_data\iChall...      None     False   
V0098  C:\Users\Prinzessin\projects\image_data\iChall...      None     False   
V0086  C:\Users\Prinzessin\projects\image_data\iChall...      None     False   
V0172  C:\Users\Prinzessin\projects\image_data\iChall...      None     False   
V0275  C:\Users\Prinzessin\projects\image_data\iChall...      None     False   

        mode  weight  
V0277  train    